In [1]:
from frontend.database import session
from scripts import generate_fc_layer_circuit, benchmark_utils
from scripts.circuit import circuit
from scripts.particles import variables, enc_vec
from scripts.reusable_modules import oneb_adder, nb_adder
import numpy as np

In [2]:
from scripts.functions import reduce_add
inp = enc_vec(name='in', nb=4)
print inp.get_variables()
out = enc_vec(name='out', nb=3)
ra = reduce_add('adder', inp, out)
ra_obj = circuit('reduce_add', ra)
ra_obj.write_file(filename='./test.sheep')

['in0C898', 'in19E40', 'in21852', 'in388C0']


In [3]:
print(inp.get_variables())
print(ra)
print(out.get_variables())

['in0C898', 'in19E40', 'in21852', 'in388C0']
in0C898 in19E40 ADD adder_red_out_1_adder_02b_adder_0_tmp12
adder_red_out_1_adder_02b_adder_0_tmp12 ALIAS adder_red_out_100AA37B1F40
in0C898 in19E40 MULTIPLY adder_red_out_1_adder_02b_adder_0_tmp29
adder_red_out_1_adder_02b_adder_0_tmp29 ALIAS adder_red_out_1_adder_0_carry_1FF
adder_red_out_1_adder_0_carry_1FF ALIAS adder_red_out_111056B3B4CD
in21852 in388C0 ADD adder_red_out_1_adder_12b_adder_0_tmp1C
adder_red_out_1_adder_12b_adder_0_tmp1C ALIAS adder_red_out_12C1EFD88089
in21852 in388C0 MULTIPLY adder_red_out_1_adder_12b_adder_0_tmp23
adder_red_out_1_adder_12b_adder_0_tmp23 ALIAS adder_red_out_1_adder_1_carry_13C
adder_red_out_1_adder_1_carry_13C ALIAS adder_red_out_135BDE697086
adder_red_out_100AA37B1F40 adder_red_out_12C1EFD88089 ADD adder_red_out_2_adder_02b_adder_0_tmp19
adder_red_out_2_adder_02b_adder_0_tmp19 ALIAS adder_red_out_203583C21F86
adder_red_out_100AA37B1F40 adder_red_out_12C1EFD88089 MULTIPLY adder_red_out_2_adder_02b_adder

In [6]:
import math
nb = 191
depth = int(math.floor(math.log(nb, 2)))
samples=3
inp = enc_vec(name='in', nb=nb)
out = enc_vec(name='out', nb=depth+1)
ra = reduce_add('adder', inp, out)
ra_obj = circuit('reduce_add', ra)
ra_obj.write_file(filename='./test.sheep')
processing_time = np.zeros(samples)
for idx in range(0, samples):
    inp_arr_val = list(map(lambda x : int(x), np.random.uniform(size=nb)> 0.5))
    inp_dict = inp.get_input_dict(inp_arr_val)
    inputs_file = benchmark_utils.write_inputs_file(inp_dict)
    results = benchmark_utils.run_circuit('./test.sheep',inputs_file,"bool","TFHE")
    processing_time[idx] = results['Processing times (s)']['circuit_evaluation']
    out_vars = out.get_variables()
    err_str = str(''.join(results['Outputs'][var] for var in out_vars)[::-1]) +" -- "+ str(np.asarray(inp_arr_val).sum())
    assert int(''.join(results['Outputs'][var] for var in out_vars)[::-1], 2) == np.asarray(inp_arr_val).sum(), err_str
print(processing_time.mean())

35.0
